# La *back-propagation*

Comme la plupart des algorithmes de ML, les réseaux de neurones sont entrainés par descente du gradient de leur fonction de coût dans l'espace des paramètres à estimer. A chaque étape, pourvu que le critère de convergence ne soit pas respecté, chaque poids du réseau $w$ est updaté suivant la formule classique (*learning rate* de $\alpha$): 

$$w \leftarrow w - \alpha.\frac{\partial J}{\partial w}$$

La *back propagation* est un l'algorithme généralement utilisé pour calculer $\frac{\partial J}{\partial w}$ pour chaque coefficient $w$ du réseau. Il n'est qu'une étape de l'algorithme d'*update* des poids du réseau qui a lieu une fois par *batch*. A la $n^e$ *batch*:
* On calcule pour chaque élément $x_i$ de la *batch* la sortie $\hat{y}_i$ associée (étape appelée *forward propagation*).
* On calcule l'erreur totale $J(y,\hat{y})$ associée, si le critère de convergence n'est pas rempli:
    * Pour l'ensemble des poinds $w$ du réseau, on calcule $\frac{\partial J}{\partial w}$ par *back propagation*.
    * On procède à l'*update* des poids du réseau.
    * On répète.
    
## L'algorithme
La *back propragation* se repose sur l'usage de la *chain rule*. Dans le cas le plus simple, celui du MLP (mais l'idée reste toujours la même), on procède en deux utilisation successives de la *chain rule*. 

La première consiste à faire apparaître la dépendance de $\frac{\partial J}{\partial w}$ à la dérivée de $J$ par rapport à l'activation de la couche courante. Cela s'écrit pour $w_{ij}^{[l]}$, $j^e$ coefficient de la fonction d'activation de la $i^e$ unité de la couche $l$ du reseau:

$$
\begin{aligned}
\frac{\partial J}{\partial w_{ij}^{[l]}} &= \frac{\partial J}{\partial a_{i}^{[l]}}.\frac{\partial a_{i}^{[l]}}{\partial w_{ij}^{[l]}} \\
&= \frac{\partial J}{\partial a_{i}^{[l]}}.a_{j}^{[l-1]}.\sigma^{'}(z_{i}^{[l]})
\end{aligned}
$$

Le second terme $\frac{\partial a_{i}^{[l]}}{\partial w_{ij}^{[l]}}$ est facile à calculer et est égal à $a_{j}^{[l-1]}.\sigma^{'}(z_{i}^{[l]})$ avec $z_{i}^{[l]}=w_{i}^{\top}.a^{[l]}+b_{i}$. Certains ont déjà été calculés lors de la *forward propagation*, le calcul de ces termes peut donc être largement accélérée par de la mise en cache. 

**Remarque**: Le procédé est le même pour tous les coefficients, y compris les biais $b_{ij}$ pour lesquels les expressions ci-dessous se simplifient un peu.

Seconde utilisation de la *chain rule*: le premier terme $\frac{\partial J}{\partial a_{i}^{[l]}}$ peut à nouveau être décomposé. 

Plutôt que de parler de *chain rule*, il est sûrement plus exact de dire qu'on décompose en différenciant $J$ par rapport à $a_{i}^{[l]}$ après avoir remarqué que $J$ dépend intégralement des activations de la couche $l+1$ (qui elles-mêmes dépendent de celle de la couche courante $l$).

Pour une fonction multivariée $f(x,y,z)$ par exemple, où chacune des variables $x$, $y$ et $z$ dépend elle-même d'une variable $t$, la différentielle de $f$ par rapport à $t$ peut en effet s'écrire: 

$$\frac{\partial f}{\partial t}=\frac{\partial f}{\partial x}.\frac{\partial x}{\partial t}+\frac{\partial f}{\partial y}.\frac{\partial y}{\partial t}+\frac{\partial f}{\partial z}.\frac{\partial z}{\partial t}$$

La dérivée $\frac{\partial J}{\partial a_{i}^{[l]}}$ peut ainsi s'écrire:

$$
\begin{aligned}
\frac{\partial J}{\partial a_{i}^{[l]}} &= \Sigma_{k=1}^{n^{[l+1]}}\frac{\partial J}{\partial a_{k}^{[l+1]}}.\frac{\partial a_{k}^{[l+1]}}{\partial a_{i}^{[l]}} \\
&= \Sigma_{k=1}^{n^{[l+1]}}\frac{\partial J}{\partial a_{k}^{[l+1]}}.w_{ki}^{[l+1]}.\sigma^{'}(z_{k}^{[l+1]})
\end{aligned}
$$

Là encore une partie des termes est soit facilement calculable, soit déjà calculée à la *forward propagation*.

Dans le cas où $l+1 = L$, c'est à dire que la couche suivante est la dernière couche du réseau, on a simplement $\frac{\partial J}{\partial a_{k}^{[l+1]}} = \frac{\partial J}{\partial \hat{y}}$ qui est simple à calculer.

### En résumé
On a montré que $\frac{\partial J}{\partial w_{ij}^{[l]}}$ pouvait s'écrire après deux utilisations de la *chain rule*:

$$\frac{\partial J}{\partial w_{ij}^{[l]}}=a_{j}^{[l-1]}.\sigma^{'}(z_{i}^{[l]}).\Sigma_{k=1}^{n^{[l+1]}}\frac{\partial J}{\partial a_{k}^{[l+1]}}.w_{ki}^{[l+1]}.\sigma^{'}(z_{k}^{[l+1]})$$

On a donc réussi à exprimer $\frac{\partial J}{\partial w_{ij}^{[l]}}$ comme une fonction: 
* De termes déjà connus de la *forward propagation* ou faciles à calculer
* Des gradients de la fonction d'erreur $J$ par rapport aux activations de la couche suivante. 

On a aussi montré qu'il existait d'une couche à l'autre, **une relation de récurrence** entre ces gradients: 

$$\frac{\partial J}{\partial a_{i}^{[l]}} = \Sigma_{k=1}^{n^{[l+1]}}\frac{\partial J}{\partial a_{k}^{[l+1]}}.w_{ki}^{[l+1]}.\sigma^{'}(z_{k}^{[l+1]})$$

Avec comme condition limite (si $l+1$ est la dernière couche): $\frac{\partial J}{\partial a_{k}^{[l+1]}} = \frac{\partial J}{\partial \hat{y}}$.

On peut ainsi en précédant à rebours (*backward*) de couche en couche à partir de la dernière du réseau, calculer successivement les gradients de $J$ par rapport à toutes les activations et donc du même coup ceux par rapport aux coefficients. A chaque étape, à chaque couche $l$, on cherche à calculer les gradients de la fonction d'erreur $J$ par rapport aux activations de la couche courante. On se sert pour cela des gradients calculés pour la couche suivante $l+1$ (rappel: on procède à rebours). Les gradients finalement calculés pour la couche $l$ sont ensuite "propagés", "passés" à la couche $l-1$ où ils serviront à calculer les gradients de $J$ par rapport aux activations de celle-ci et ainsi de suite.

## *Vanishing & exploding gradients*
Pour chaque couche $l$, les gradients calculés sont une combinaison linéaire des gradients calculés à la couche suivante $l+1$. La répétition de ces combinaisons linéraires peut avoir pour conséquence une certaine **instabilité numérique** des réseaux de neurones présentant un nombre important de couches (et donc d'étapes de *back-propagation*), i.e. profonds. Les gradients qui se propagent peuvent en effet devenir de plus en plus petits jusqu'à devenir négligeables (*vanishing*) ou à l'inverse devenir très grands (*exploding*). 

Dans les deux cas c'est problématique: 
* En cas de *vanishing gradients*, les ajustements opérés sur les coefficients des premières couches du réseau sont négligeables. Ces couches apparaissent alors comme très difficiles à entrainer.
* En cas d'*exploding gradients*, les ajustements opérés sont à l'inverse très importants et l'entrainement ne converge pas.

Dans le cas de l'*exploding gradients*, différentes solutions existent:
* La solution simple et efficace est le *gradient clipping*. On fixe un ajustement maximal des coefficients: si la norme du gradient dépasse un certain seuil, l'ajustement se fera dans la même direction que calculée mais avec un pas de la taille du seuil.
* Empêcher les poids de devenir trop gros en les pénalisant dans la fonction de coût (régularisation). On agit sur la valuer des poids dont ont a vu qu'ils intervenaient directement dans le calcul de $\frac{\partial J}{\partial w_{ij}^{[l]}}$. Globalement il n'y a que les poids qui peuvent devenir grand dans l'expression de $\frac{\partial J}{\partial w_{ij}^{[l]}}$: les dérivées des fonctions d'activation sont bornée et les activations aussi.

Dans le cas des *vanishing gradients*:
* Changer de fonction d'activation, privilégier les fonctions d'activation dont la dérivée ne peut pas devenir trop petite. Ex: ReLU dans le domaine positif. On agit sur $\sigma^{'}$ dont ont a vu qu'elle intervient directement dans le calcul de $\frac{\partial J}{\partial w_{ij}^{[l]}}$.
* Itervenir sur l'architecture du réseau: introduire des *residual connections*.


## Cas des RNN (*back-propagation through time*)
La *back-propagation* dans le cas des RNN peut pour certains coefficients être à la fois source d'instabilité numérique et se révéler trop couteuse. 

Prenons le cas des RNN simples dans lesquels d'état caché à l'étape $t$ se calcule simplement par la relation $h_t=f(x_t,h_{t-1})$, on a trois matrices de coefficients à estimer (sans compter autant de vecteurs de biais):
* $W_{hh}$ et $W_{xh}$ dont dépendent le calcul de l'activation de la couche RNN elle-même (i.e.: l'état caché).
* $W_{ho}$ dont dépend le calcul de l'activation de la couche d'*output* à chaque étape à partir de l'état caché de la même étape.

Deux particularités des RNN:
* La *loss* globale $J$ s'écrit comme la somme des *losses* à chaque étape $J^{[t]}$. Pour chaque coefficient à *updater*, l'update va s'exprimer en fonction des gradients des *losses* à chacune des étapes. Dit autrement: on doit *back-propagate* les variations non pas d'une *loss* mais de $T$ *step losses*.
* Pour les coefficients de $W_{hh}$ et $W_{xh}$, un problème apparaît au niveau de la première *chain rule* du calcul de la *back-propagation* présenté plus haut, pour le calcul du terme $\frac{\partial a_{i}^{[l]}}{\partial w_{ij}^{[l]}}$ (et ici: $w_{ij}^{[l]}=w_{ij}$ indépendant de la couche par construction). Dans le cas d'un RNN, la $i^e$ activation $a_{i}^{[l]}$ de la couche $l$ correspond à la $i^e$ composante $h_{i}^{[l]}$ de l'état caché produit à la $l^e$ étape. Or par la relation récursive $h_t=f(x_t,h_{t-1})$, $h^{[l]}$ dépend de tous les états cachés antérieurs qui eux même dépendent tous du coefficient $w_{ij}$ qu'on cherche à *updater*. 

On va devoir calculer pour chaque $J^{[t]}$, tous les $\frac{\partial J^{[t]}}{\partial h_{i}^{[l]}}$ (avec $1 \leq l \leq t$), ce qui revient à *back propagate* un gradient $t$ fois à travers la couche RNN: pour $J^{[1]}$, le gradient par rapport à l'état caché sera *back-propagated* une fois à travers la couche RNN, 2 fois pour $J^{[2]}$, 3 fois pour $J^{[3]}$, etc.

Dit autrement, en plus de devoir *back-propagate* $T$ *loss gradients*, chaque *loss gradient* doit être *back-propagated* d'autant plus loin qu'il correspond à un step $t$ avancé. Au delà du coût computationnel élevé, pour $t$ élevé, on retombe dans le problème d'instabilité numérique inhérents aux réseaux profonds.

Une solution simple et qui s'est prouvée efficace consiste à **tronquer** (*truncate*) la propagation du gradient: à partir d'un certain nombre de *back-propagations* à travers la couche RNN $\tau$, on considère la contribution des gradients négligeable. Dit autrement: on fait l'hypothèse que les variations de la *loss* à l'étape $t$ est intégralement expliquée par les variations d'activation entre les étapes $t-\tau$ et $t$. Antérieurement à l'étape $\tau$, les variations d'activation n'ont qu'un impact négligeable sur les variations de la *loss* à l'étape $t$. La troncature revient finalement à faire **l'hypothèse régularisante** que seuls les "éléments récents" (i.e. les $\tau$ derniers *inputs*) ont un impact significatif sur les performances. Ce faisant, on acte le fait que les RNN dans leur forme simple ne parviennent pas à faire "circuler de l'information" sur des longues durées (*short-term memory*) et donc à apprendre les comportements de long terme de la séquence (constater des *vanishing gradients* est déjà un signe que l'information ne peut se propager que de façon limitée). Les architectures alternatives de couche RNN (GRU, LSTM) permettent de partiellement répondre au problème mais restent in fine soumises au même type de limitation de *vanishing gradient* (elles peuvent voir un peu plus loin, d'où le terme de *long short-term memory* mais pas beaucoup plus loin non plus).   